In [ ]:
import requests
import os

from datetime import datetime
import pandas as pd

import pygsheets
from pygsheets.datarange import DataRange
from string import ascii_uppercase

In [ ]:
api_token = os.environ.get("TILTIFY_TOKEN")
campaign_id = 155357

In [ ]:
polls = {}

response = requests.get(
    f"https://tiltify.com/api/v3/campaigns/{campaign_id}/polls",
    headers={"Authorization": "Bearer {}".format(api_token)},
)

for row in response.json()["data"]:
    polls[row["id"]] = row

In [ ]:
finished_refreshing = datetime.now()

In [ ]:
polls

In [ ]:
polls_rows = []

for poll in polls.values():
    for option in poll["options"]:
        polls_rows.append(
            {
                "Question": poll["name"],
                "Created": datetime.utcfromtimestamp(poll["createdAt"] / 1000),
                "Finished": datetime.utcfromtimestamp(poll["updatedAt"] / 1000),
                "Total Raised": sum(x["totalAmountRaised"] for x in poll["options"]),
                "Option": option["name"],
                "Raised": option["totalAmountRaised"],
            }
        )

In [ ]:
polls_raised = pd.DataFrame(polls_rows).sort_values("Finished").reset_index(drop=True)
polls_raised["Finished"] = polls_raised["Finished"].astype("str")
polls_raised["Created"] = polls_raised["Created"].astype("str")
polls_raised

In [ ]:
polls_raised = polls_raised[polls_raised["Created"] != polls_raised["Finished"]]
polls_raised = polls_raised[polls_raised["Total Raised"] != 0]
polls_raised

In [ ]:
def get_sheet(document, sheet_name):
    return document.worksheets("title", sheet_name)[0]

In [ ]:
gc = pygsheets.authorize()

In [ ]:
document_url = os.environ.get(
    "DOCUMENT_URL",
    "https://docs.google.com/spreadsheets/d/1q2i2dycsXRrYHofHXNCL9q1XtTxmTQYo2bUyxvStHmI/edit#gid=0",
)

In [ ]:
document = gc.open_by_url(document_url)
sheet = get_sheet(document, "Polls")

In [ ]:
sheet.clear(fields="*")
for i in range(1, 11):
    sheet.adjust_column_width(start=i, end=i, pixel_size=100)
rng = sheet.get_values("A1", "I2000", returnas="range")
rng.merge_cells("NONE")

In [ ]:
sheet.update_values("A1", [list(polls_raised.columns)])
values = polls_raised.values.tolist()
values = [[None if pd.isna(x) else x for x in row] for row in values]
sheet.update_values("A2", values)

In [ ]:
position = len(polls_raised.columns) + 1
sheet.update_value(f"{ascii_uppercase[position]}1", "Last updated")
sheet.adjust_column_width(start=position + 1, end=position + 1, pixel_size=100)
sheet.update_value(f"{ascii_uppercase[position + 1]}1", datetime.now().isoformat())
sheet.adjust_column_width(start=position + 2, end=position + 2, pixel_size=150)

In [ ]:
def set_number_formatting(sheet, column, format_type, pattern):
    model_cell = pygsheets.Cell("A1")

    model_cell.set_number_format(format_type=format_type, pattern=pattern)

    pygsheets.DataRange(column, column, worksheet=sheet).apply_format(model_cell)

In [ ]:
formatting = {
    "Total Raised": (pygsheets.FormatType.NUMBER, "0.00"),
    "Raised": (pygsheets.FormatType.NUMBER, "0.00"),
}

In [ ]:
for column, (fmt, pattern) in formatting.items():
    set_number_formatting(
        sheet,
        ascii_uppercase[list(polls_raised.columns).index(column)],
        fmt,
        pattern,
    )

In [ ]:
header_range = DataRange(
    start="A1",
    end=f"{ascii_uppercase[len(polls_raised.columns) - 1]}1",
    worksheet=sheet,
)
header_range.update_borders(bottom=True, style="SOLID")

In [ ]:
for column_name in polls_raised.columns:
    column = ascii_uppercase[list(polls_raised.columns).index(column_name)]
    header_cell = sheet.cell(f"{column}1").set_text_format("bold", True)
    if column_name in formatting.keys():
        header_cell.set_horizontal_alignment(
            pygsheets.custom_types.HorizontalAlignment.RIGHT
        )
    header_cell.update()

In [ ]:
widths = {
    "Question": 500,
    "Created": 140,
    "Finished": 140,
    "Option": 400,
}

In [ ]:
for column, width in widths.items():
    sheet.adjust_column_width(
        start=list(polls_raised.columns).index(column) + 1,
        end=list(polls_raised.columns).index(column) + 1,
        pixel_size=width,
    )

In [ ]:
for question in polls_raised["Question"].unique():
    range_df = polls_raised[polls_raised["Question"] == question]
    index_start, index_end = range_df.index.min() + 2, range_df.index.max() + 2
    print(question, index_start, index_end)

    for x in ["A", "B", "C", "D"]:
        rng = sheet.get_values(f"{x}{index_start}", f"{x}{index_end}", returnas="range")
        rng.merge_cells()
        cell = sheet.get_values(
            f"{x}{index_start}", f"{x}{index_end}", returnas="cell"
        )[0][0]

        cell.set_vertical_alignment(pygsheets.custom_types.VerticalAlignment.MIDDLE)

In [ ]:
for column_name in polls_raised.columns:
    column = ascii_uppercase[list(polls_raised.columns).index(column_name)]
    if column_name in {"Created", "Finished"}:
        rng = sheet.get_values(column, column, returnas="cells")
        for row in rng:
            if row[0].value == "":
                continue

            row[0].set_horizontal_alignment(
                pygsheets.custom_types.HorizontalAlignment.RIGHT
            )